In [1]:
import os
import shutil
from collections import Counter
import numpy as np
import json
import torch
import torch.nn as nn
import torch.nn.functional as F
from transformers import AutoTokenizer, ElectraForQuestionAnswering, DataCollatorWithPadding,BertModel, ElectraForSequenceClassification, ElectraModel
from Preprocess.arabertpreprocess import ArabertPreprocessor
import matplotlib.pyplot as plt
import seaborn as sns
import csv
torch.manual_seed(3407)

## Preprocessing

In [2]:
def add_end_index(answer, context):
  ## 1 if span match the context 0 otherwise
  text = answer['text']
  start_idx = answer['answer_start']
  end_idx = start_idx + len(text)
  answer['answer_end'] = end_idx
  if text == context[start_idx:end_idx]:
    answer['answer_end'] = end_idx
    return False
  for i in range(1,3):
    if text == context[start_idx-i:end_idx-i]:
      answer['answer_end']= end_idx-1
      answer['answer_start'] = start_idx-1
      return False
  return True

In [3]:
def arabert_preprocess(context,question, answer, arabert_prep):
    answer['text'] = arabert_prep.preprocess(answer['text'])
    context = arabert_prep.preprocess(context)
    question = arabert_prep.preprocess(question)
    res = context.find(answer['text'])
    if res !=-1:
        answer['answer_start'] = res
    return context, question, answer, res

In [4]:
def Read_AAQAD(path,arabert_prep):
  contexts =[]
  answers =[]
  questions =[]
  IDs= []
  plausible = []
  cnt = 0
  with open(path) as f:
    aaqad_dict = json.load(f)
    for article in aaqad_dict['data']:
      for passage in article['paragraphs']:
        context = passage['context']
        for qa in passage['qas']:
          question = qa['question']
          if 'plausible_answers' in qa.keys():# there is two cases if the question have no answer then use plausible answer
            access = 'plausible_answers'
            plausible.append(True)
          else:
            access = 'answers'
            plausible.append(False)
          for answer in qa[access]:
            context,question, answer, res =  arabert_preprocess(context,question, answer, arabert_prep)
            #if res==-1:
            #  cnt+=1
            #  continue
            flag = add_end_index(answer, context) #if false dont add the 
            cnt =cnt + flag
            flag = False
            if not flag:
              contexts.append(context)
              answers.append(answer)
              questions.append(question)
              IDs.append(int(qa['id']))
  return contexts,questions,answers,plausible,IDs

In [5]:
def fix_ids(path):
  #IDs need to be fixed for evaluating purposes
    a_file = open(path, "r")
    json_object = json.load(a_file)
    a_file.close()
    idx_cnt = 1
    for article in json_object['data']:
      for passage in article['paragraphs']:
        context = passage['context']
        for qa in passage['qas']:
            qa['id'] = str(idx_cnt)
            idx_cnt = idx_cnt + 1
    a_file = open(path, "w")
    json.dump(json_object, a_file)
    a_file.close()

In [6]:
model_name = "araelectra-base-discriminator"
arabert_prep = ArabertPreprocessor(model_name=model_name)
fix_ids('Data/asquadv2-train.json')
fix_ids('Data/asquadv2-val.json')
fix_ids('Data/asquadv2-test.json')
asquad_train_contexts, asquad_train_questions, asquad_train_answers,asquad_train_plausible, asquad_train_ids = Read_AAQAD('Data/asquadv2-train.json', arabert_prep)
asquad_val_contexts, asquad_val_questions, asquad_val_answers,asquad_val_plausible, asquad_val_ids = Read_AAQAD('Data/asquadv2-val.json', arabert_prep)
asquad_test_contexts, asquad_test_questions, asquad_test_answers,asquad_test_plausible, asquad_test_ids = Read_AAQAD('Data/asquadv2-test.json', arabert_prep)


In [7]:
len(asquad_test_answers)+len(asquad_train_answers)+len(asquad_val_answers)

96051

In [8]:
print(len(asquad_test_answers))
print(len(asquad_val_answers))
print(len(asquad_train_answers))

9606
9605
76840


In [10]:
sum(asquad_test_plausible)/len(asquad_test_plausible) #shows the percentage of not answerd questions

0.452841973766396

In [7]:
print(sum(asquad_train_ids)==len(asquad_train_ids)*(len(asquad_train_ids)+1)/2)
print(sum(asquad_val_ids)==len(asquad_val_ids)*(len(asquad_val_ids)+1)/2)
print(sum(asquad_test_ids)==len(asquad_test_ids)*(len(asquad_test_ids)+1)/2)

True
True
True


In [8]:
def get_answered_feat(contexts, questions, answers, plausible):
    new_contexts, new_questions, new_answers = [], [], []
    for i in range(len(answers)):
        if plausible[i] == False:
            new_contexts.append(contexts[i])
            new_questions.append(questions[i])
            new_answers.append(answers[i])
    return new_contexts, new_questions, new_answers
span_train_contexts, span_train_questions, span_train_answers = get_answered_feat(asquad_train_contexts, asquad_train_questions, asquad_train_answers, asquad_train_plausible)   

In [9]:
print(len(span_train_contexts), len(asquad_train_contexts))
print(sum(asquad_test_plausible))

42042 76840
4350


## Tokenization

In [10]:
#Creating the tokenizer
model_name =  "aubmindlab/araelectra-base-discriminator"
araelectra_tokenizer = AutoTokenizer.from_pretrained(model_name,do_lower_case=False)

In [11]:
train_encodings = araelectra_tokenizer(asquad_train_questions, asquad_train_contexts, truncation = True )
span_train_encodings = araelectra_tokenizer(span_train_questions, span_train_contexts, truncation=True)
val_encodings = araelectra_tokenizer(asquad_val_questions, asquad_val_contexts, truncation=True, return_offsets_mapping=True)
test_encodings = araelectra_tokenizer(asquad_test_questions, asquad_test_contexts, truncation=True,  return_offsets_mapping=True)

In [12]:
val_offset = val_encodings['offset_mapping']
del val_encodings['offset_mapping']
test_offset = test_encodings['offset_mapping']
del test_encodings['offset_mapping']

In [13]:
val_ids_to_idx = {k:i for i,k in enumerate(asquad_val_ids)}
test_ids_to_idx = {k:i for i,k in enumerate(asquad_test_ids)}


In [14]:
def index_to_token_position(encodings , answers):
  start_positions = list()
  end_positions = list()
  for i in range(len(answers)):
    start_positions.append(encodings.char_to_token(i, answers[i]['answer_start'], 1))
    end_positions.append(encodings.char_to_token(i, answers[i]['answer_end'], 1))
    #if context truncated
    if start_positions[-1] is None: 
      start_positions[-1] = araelectra_tokenizer.model_max_length
    #if end index is space
    itt = 1
    while end_positions[-1] is None: 
      end_positions[-1] = encodings.char_to_token(i, answers[i]['answer_end']-itt, 1)
      itt = itt + 1 
  encodings.update({'start_positions': torch.tensor(start_positions), 'end_positions': torch.tensor(end_positions)})
  encodings['start_positions'] = encodings['start_positions'].view(len(answers), 1)
  encodings['end_positions'] = encodings['end_positions'].view(len(answers), 1)

In [15]:
index_to_token_position(span_train_encodings, span_train_answers)
index_to_token_position(val_encodings, asquad_val_answers)
index_to_token_position(test_encodings, asquad_test_answers)

In [16]:
def add_weights_labels_tensors(encodings, plausible):
  plausible = torch.tensor(plausible)
  weights = torch.ones(plausible.shape)
  no_ans = torch.ones(plausible.shape)
  weights[plausible==False]=2.0
  no_ans[plausible==False]=0.0
  weights = weights.view(-1,1)
  no_ans = no_ans.view(-1,1)
  encodings.update({'weights':weights, 'no_ans':no_ans})

In [17]:
add_weights_labels_tensors(train_encodings, asquad_train_plausible)
add_weights_labels_tensors(val_encodings, asquad_val_plausible)
add_weights_labels_tensors(test_encodings, asquad_test_plausible)

In [18]:
val_encodings['IDs'] = asquad_val_ids
test_encodings['IDs'] = asquad_test_ids

In [19]:
print(train_encodings.keys())
print(val_encodings.keys())
print(test_encodings.keys())
print(span_train_encodings.keys())

dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'weights', 'no_ans'])
dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'start_positions', 'end_positions', 'weights', 'no_ans', 'IDs'])
dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'start_positions', 'end_positions', 'weights', 'no_ans', 'IDs'])
dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'start_positions', 'end_positions'])


## Dataset and DataLoader

In [20]:
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from tqdm import tqdm

In [21]:
class AqadDataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings
    def __getitem__(self, idx):
        return {key: val[idx] for key, val in self.encodings.items()}

    def __len__(self):
        return len(self.encodings.input_ids)

cls_train_dataset = AqadDataset(train_encodings)
span_train_dataset = AqadDataset(span_train_encodings)
val_dataset = AqadDataset(val_encodings)
test_dataset = AqadDataset(test_encodings)

In [22]:
data_collator = DataCollatorWithPadding(araelectra_tokenizer)

In [23]:
cls_train_loader = DataLoader(cls_train_dataset, batch_size=8, shuffle= True, collate_fn= data_collator)
span_train_loader = DataLoader(span_train_dataset, batch_size=8, shuffle= True, collate_fn = data_collator)
val_loader = DataLoader(val_dataset, batch_size = 8, shuffle = True, collate_fn = data_collator)
test_loader = DataLoader(test_dataset, batch_size = 8, shuffle = True, collate_fn = data_collator)


## Checkpoints

In [24]:
def save_ckp(state, is_best, checkpoint_path, best_model_path):
    """
    state: checkpoint to save
    is_best: is this the best checkpoint; min validation loss
    checkpoint_path: path to save checkpoint
    best_model_path: path to save best checkpoint
    """
    f_path = checkpoint_path
    # save checkpoint data to the path given, checkpoint_path
    torch.save(state, f_path)
    # if it is a best model, min validation loss
    if is_best:
        best_fpath = best_model_path
        # copy that checkpoint file to best path given, best_model_path
        shutil.copyfile(f_path, best_fpath)

In [25]:
def load_ckp(checkpoint_fpath, model, optimizer):
    """
    checkpoint_path: path to saved checkpoint
    model: model to load checkpoint parameters into       
    optimizer: optimizer defined in previous training
    """
    # load check point
    checkpoint = torch.load(checkpoint_fpath)
    # initialize state_dict from checkpoint to model
    model.load_state_dict(checkpoint['state_dict'])
    # initialize optimizer from checkpoint to optimizer
    optimizer.load_state_dict(checkpoint['optimizer'])
    # initialize valid_loss_min from checkpoint to valid_loss_min
    results = checkpoint['result_dict']
    # return model, optimizer, epoch value, min validation loss 
    return model, optimizer, checkpoint['epoch'], results

In [26]:
def order_exp(base_path, exp_name):
  exp_path = os.path.join(base_path, exp_name)
  if not os.path.exists(exp_path):
    os.mkdir(exp_path)
  curr_ckp_path = os.path.join(exp_path,'curr.pt')
  best_ckp_path = os.path.join(exp_path, 'best.pt')
  return curr_ckp_path, best_ckp_path, exp_path

## Classification train and evaluation 

In [27]:
def cls_eval(model, data_loader, exp_path, train_loss):
    model.eval()
    total_acc = 0
    total_pred = None
    total_IDs = None
    soft = torch.nn.Softmax(dim=1)
    with open(os.path.join(exp_path,'preds.txt'),'w') as f:
        pass
    for batch in data_loader:
      tokens = batch['input_ids'].to(device)
      masks = batch['attention_mask'].to(device)
      tokens_type = batch['token_type_ids'].to(device)
      IDs = batch['IDs'].to(device)
      gt_no_ans = batch['no_ans'].to(device)
      output = model(tokens, masks, tokens_type)
      pred = output.logits.view(masks.shape[0],2,)
      with open(os.path.join(exp_path,'preds.txt'),'a') as f:
        preds_soft = soft(pred)[:,1]
        IDs
        for i in range(preds_soft.shape[0]):
            f.write(f"{IDs[i].item()},{preds_soft[i].item()}\n")
        
      pred = torch.argmax(pred, dim=1)
      target = batch['no_ans'].to(device).view(masks.shape[0],)
      total_acc += torch.sum(target==pred)

    total_acc = total_acc/ val_dataset.__len__()
    res_dict = {'acc':total_acc.item()*100, 'train_loss':train_loss}
    if exp_path:
        log_path = os.path.join(exp_path,'res.csv')
        if not os.path.exists(log_path):
            with open(log_path,'w') as f:
                writer = csv.DictWriter(f, fieldnames=res_dict.keys())
                writer.writeheader()
        with open(log_path, 'a') as f:
            writer = csv.DictWriter(f, fieldnames=res_dict.keys())
            #writer.writeheader()
            writer.writerow(res_dict)
    model.train()
    return res_dict


In [28]:
def cls_train(model,start_epoch, num_epochs, optimizer,max_acc, train_loader, val_loader, log, exp_name):
  curr_ckp_path, best_ckp_path, exp_path = order_exp('Runs/AraElectraDecoupledAsquadv2/train/cls', exp_name)
  model.train()
  cls_pred = None
  for epoch in range(start_epoch,num_epochs):
    total_loss = 0.0
    loop = tqdm(train_loader, leave=True)
    for batch_idx, batch in enumerate(loop):
      tokens = batch['input_ids'].to(device)
      masks = batch['attention_mask'].to(device)
      tokens_type = batch['token_type_ids'].to(device)
      weights = batch['weights'].to(device)
      output = model(tokens, masks, tokens_type)
      pred = output.logits.view(masks.shape[0],2,)
      target = batch['no_ans'].type(torch.LongTensor)
      target = target.to(device)
      loss = cls_criterion(pred, target.view(masks.shape[0],))
      loss = loss*(weights.view(masks.shape[0],))
      loss = torch.mean(loss)
      loss.backward()
      optimizer.step()
      optimizer.zero_grad()
      total_loss = total_loss + ((1 / (batch_idx + 1)) * (loss.item() - total_loss)) 
      loop.set_description(f'Epoch {epoch}')
      loop.set_postfix(loss=loss.item())

    result_dict= cls_eval(model, val_loader,exp_path,total_loss )
    checkpoint = {
            'epoch': epoch + 1,
            'result_dict':result_dict,
            'state_dict': model.state_dict(),
            'optimizer': optimizer.state_dict(),
        }
    curr_acc = result_dict['acc']
    if curr_acc>=max_acc:
      max_acc = curr_acc
      save_ckp(checkpoint, True, curr_ckp_path, best_ckp_path)
    else:
      save_ckp(checkpoint, False, curr_ckp_path, best_ckp_path)
    print(result_dict)
  return model


## Modeling

In [29]:
Cls_AraElectra = ElectraForSequenceClassification.from_pretrained(model_name, num_labels=2)
QA_AraElectra = ElectraForQuestionAnswering.from_pretrained(model_name)

Some weights of the model checkpoint at aubmindlab/araelectra-base-discriminator were not used when initializing ElectraForSequenceClassification: ['discriminator_predictions.dense.weight', 'discriminator_predictions.dense.bias', 'discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense_prediction.bias']
- This IS expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at aubmindlab/araelectra-base-discriminator and are newly initialized: 

In [30]:
def freeze(Electra, count=None):
    if count is not None:
	      # We freeze here the embeddings of the model
        for param in Electra.embeddings.parameters():
            param.requires_grad = False

        if count != -1:
	          # if freeze_layer_count == -1, we only freeze the embedding layer
	          # otherwise we freeze the first `freeze_layer_count` encoder layers
            for layer in Electra.encoder.layer[:count]:
                for param in layer.parameters():
                    param.requires_grad = False
    print(sum(p.numel() for p in Electra.parameters()), sum(p.numel() for p in Electra.parameters() if p.requires_grad))

In [31]:
freeze(Cls_AraElectra.electra,6)
freeze(QA_AraElectra.electra, 6)

134602752 42527232
134602752 42527232


## Classification Training

In [32]:
num_epochs = 2
learning_rate = 3e-5
optimizer = torch.optim.Adam(Cls_AraElectra.parameters(), lr=learning_rate)
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
#criterion_span = nn.CrossEntropyLoss(reduction='none')
cls_criterion = nn.CrossEntropyLoss(reduction='none')
Cls_AraElectra.to(device)

ElectraForSequenceClassification(
  (electra): ElectraModel(
    (embeddings): ElectraEmbeddings(
      (word_embeddings): Embedding(64000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): ElectraEncoder(
      (layer): ModuleList(
        (0): ElectraLayer(
          (attention): ElectraAttention(
            (self): ElectraSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): ElectraSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm

In [33]:
cls_trained_model, cls_pred = cls_train(Cls_AraElectra, 0, 2, optimizer, 0, cls_train_loader, val_loader , True, 'third')

Epoch 1: 100%|██████████| 9605/9605 [1:28:55<00:00,  1.80it/s, loss=0.562] 


{'acc': 79.40655946731567, 'train_loss': 0.6688395292835084}
{'acc': 84.32066440582275, 'train_loss': 0.4508261918751091}


In [35]:
cls_trained_model = cls_train(cls_trained_model, 2, 6, optimizer, 84.3, cls_train_loader, val_loader , True, 'third')

Epoch 3:  34%|███▍      | 3242/9605 [29:53<1:15:22,  1.41it/s, loss=0.00431]

{'acc': 85.14315485954285, 'train_loss': 0.32205695654617095}


In [33]:
cls_trained_model, optimizer , x, xx = load_ckp('Runs/AraElectraDecoupledAsquadv2/train/cls/third/curr.pt',Cls_AraElectra, optimizer)

In [34]:
cls_trained_model = cls_train(cls_trained_model, 3, 8, optimizer, 85.1, cls_train_loader, val_loader , True, 'third')

Epoch 7:   2%|▏         | 174/9605 [01:35<1:26:23,  1.82it/s, loss=0.0748]  


{'acc': 84.52889323234558, 'train_loss': 0.19359704868555624}
{'acc': 84.82040762901306, 'train_loss': 0.1257312193015077}
{'acc': 85.09109616279602, 'train_loss': 0.10057395500561123}
{'acc': 84.21655297279358, 'train_loss': 0.15071689100924335}


KeyboardInterrupt: 

## Load Cls Model if needed

In [ ]:
cls_model = ElectraForSequenceClassification.from_pretrained(model_name)

In [ ]:
learning_rate = 3e-5
optimizer = torch.optim.AdamW(cls_model.parameters(), lr=learning_rate)
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
#criterion_span = nn.CrossEntropyLoss(reduction='none')
#cls_criterion = nn.CrossEntropyLoss()
cls_model.to(device)

In [ ]:
cls_model, optimizer, start_epoch, result_dict = load_ckp('Runs/AraElectraDecoupledAsquadv2/train/cls/second/best.pt', cls_model, optimizer)


## Span Training

In [29]:
def get_raw_preds(data_loader, model,ids_to_index,offset,contexts, max_answer_length, n_best_size): 
  model.eval()
  imd_predictions,script_predictions = dict(), dict()
  with torch.no_grad():
    #F1 = EM = Total = 0
    total_loss = 0.0
    total_predictions = dict()
    no_probs_pred = dict()
    #loop = tqdm(data_loader)
    loop = tqdm(data_loader, leave=True)
    for batch_idx, batch in enumerate(loop):
      tokens = batch['input_ids'].to(device)
      masks = batch['attention_mask'].to(device)
      tokens_type = batch['token_type_ids'].to(device)
      gt_start = batch['start_positions'].to(device)
      gt_end = batch['end_positions'].to(device)
      IDs = batch['IDs'].to(device)
      outputs = model(tokens, masks, tokens_type, start_positions=gt_start, end_positions=gt_end)
      #calculating loss
      loss = outputs.loss
      #update average total loss 
      total_loss = total_loss + ((1 / (batch_idx + 1)) * (loss.item() - total_loss)) 
      #calculating f1 score and EM
      curr_batch_size = tokens.shape[0]
      post_raw_preds(IDs, outputs.start_logits, outputs.end_logits, ids_to_index, offset, contexts,max_answer_length, n_best_size, imd_predictions, script_predictions )
    #saving evaluation results
    #evaluation

    model.train()
    return imd_predictions,script_predictions

In [30]:
def post_raw_preds(IDs, total_start_logits, total_end_logits,ids_to_index,offset,contexts, max_answer_length, n_best_size,
 imd_predictions,script_predictions ):
    total_start_logits = total_start_logits.cpu().numpy()
    total_end_logits = total_end_logits.cpu().numpy()
    IDs = IDs.cpu().numpy()
    for i in range(IDs.shape[0]):
        offset_mapping = offset[ids_to_index[IDs[i].squeeze()]]
        # The first feature comes from the first example. For the more general case, we will need to be match the example_id to
        # an example index
        context = contexts[ids_to_index[IDs[i].squeeze()]]
        start_logits = total_start_logits[i]
        end_logits = total_end_logits[i]
        # Gather the indices the best start/end logits:
        start_indexes = np.argsort(start_logits)[-1 : -n_best_size - 1 : -1].tolist()
        end_indexes = np.argsort(end_logits)[-1 : -n_best_size - 1 : -1].tolist()
        valid_answers = []
        for start_index in start_indexes:
            for end_index in end_indexes:
                # Don't consider out-of-scope answers, either because the indices are out of bounds or correspond
                # to part of the input_ids that are not in the context.
                if (
                    start_index >= len(offset_mapping)
                    or end_index >= len(offset_mapping)
                    or offset_mapping[start_index] is None
                    or offset_mapping[end_index] is None
                ):
                    continue
                # Don't consider answers with a length that is either < 0 or > max_answer_length.
                if end_index < start_index or end_index - start_index + 1 > max_answer_length:
                    continue
                if start_index <= end_index: # We need to refine that test to check the answer is inside the context
                    start_char = offset_mapping[start_index][0]
                    end_char = offset_mapping[end_index][1]
                    valid_answers.append(
                        {
                            "score": start_logits[start_index] + end_logits[end_index],
                            "text": context[start_char: end_char]
                        }
                    )
        if len(valid_answers) ==0:
            valid_answers.append({"text":"", "score":""})

        valid_answer = sorted(valid_answers, key=lambda x: x["score"], reverse=True)[0]
        imd_predictions[str(IDs[i].squeeze())] = valid_answer
        script_predictions[str(IDs[i].squeeze())] = valid_answer['text']

In [31]:
def get_preds(total_preds, no_probs_preds,data_path, log_path):
    preds_path = os.path.join(log_path, 'preds')
    if not os.path.exists(preds_path):
        os.mkdir(preds_path)
    no_probs_path = os.path.join(preds_path, 'na_probs.json')
    text_preds_path = os.path.join(preds_path, 'preds.json')
    jsonString = json.dumps(total_preds)
    jsonFile = open(text_preds_path, "w")
    jsonFile.write(jsonString)
    jsonFile.close()
    if no_probs_preds is not None:
        jsonString = json.dumps(no_probs_preds)
        jsonFile = open(no_probs_path, "w")
        jsonFile.write(jsonString)
        jsonFile.close()
        #!python evaluatev2.py data_path text_preds_path electra --na-prob-file no_probs_path --na-prob-thresh 0.4 --out-file log_path
        #os.system(f"python evaluatev2.py {data_path} {text_preds_path} electra --na-prob-file {no_probs_path} --na-prob-thresh 0.5 --out-file {log_path}")
        !/anaconda/envs/azureml_py38/bin/python3 evaluatev2.py Data/asquadv2-val.json Runs/AraElectraDecoupledAsquadv2/train/span/fourth/preds/preds.json electra --na-prob-file Runs/AraElectraDecoupledAsquadv2/train/span/second/preds/na_probs.json  --na-prob-thresh 0.5  
    else:
        !/anaconda/envs/azureml_py38/bin/python3 evaluatev2.py Data/asquadv2-val.json Runs/AraElectraDecoupledAsquadv2/train/span/fourth/preds/preds.json electra  --out-file Runs/AraElectraDecoupledAsquadv2/train/span/fourth
    if log_path:
        with open(os.path.join(log_path, 'res.csv')) as f:
            DictReader_obj = csv.DictReader(f)
            lastrow = None
            for item in DictReader_obj:
                lastrow = dict(item)
        #print(lastrow)
        return lastrow
    return 1


In [32]:
def span_train(model,start_epoch, num_epochs, optimizer,max_compined_metric, train_loader, val_loader, log, exp_name):
  curr_ckp_path, best_ckp_path, exp_path = order_exp('Runs/AraElectraDecoupledAsquadv2/train/span', exp_name)
  model.train()
  for epoch in range(start_epoch,num_epochs):
    total_loss = 0.0
    loop = tqdm(train_loader, leave=True)
    for batch_idx, batch in enumerate(loop):
      tokens = batch['input_ids'].to(device)
      masks = batch['attention_mask'].to(device)
      tokens_type = batch['token_type_ids'].to(device)
      gt_start = batch['start_positions'].to(device)
      gt_end = batch['end_positions'].to(device)
      outputs = model(tokens, masks, tokens_type, start_positions=gt_start, end_positions=gt_end)
      loss = outputs.loss
      loss = 2*loss
      loss.backward()
      optimizer.step()
      optimizer.zero_grad()
      total_loss = total_loss + ((1 / (batch_idx + 1)) * (loss.item() - total_loss)) 
      loop.set_description(f'Epoch {epoch}')
      loop.set_postfix(loss=loss.item())
    
    imd_preds, script_preds = get_raw_preds(val_loader, model,val_ids_to_idx,val_offset,asquad_val_contexts, 30, 10)
    result_dict = get_preds(script_preds, None,'Data/asquadv2-val.json',exp_path )
    checkpoint = {
            'epoch': epoch + 1,
            'result_dict':result_dict,
            'state_dict': model.state_dict(),
            'optimizer': optimizer.state_dict(),
        }
    curr_compined_metric = float(result_dict['HasAns_exact'])+1.5*float(result_dict['HasAns_f1'])
    if curr_compined_metric>=max_compined_metric:
      max_compined_metric = curr_compined_metric
      save_ckp(checkpoint, True, curr_ckp_path, best_ckp_path)
    else:
      save_ckp(checkpoint, False, curr_ckp_path, best_ckp_path)
    print("ckp saved")
  return model


In [35]:
QA_AraElectra = ElectraForQuestionAnswering.from_pretrained(model_name)
freeze(QA_AraElectra.electra, 6)
span_num_epochs = 4
span_learning_rate = 3e-5
span_optimizer = torch.optim.AdamW(QA_AraElectra.parameters(), lr=span_learning_rate)
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
#criterion_span = nn.CrossEntropyLoss(reduction='none')
#cls_criterion = nn.CrossEntropyLoss()
QA_AraElectra.to(device)

Some weights of the model checkpoint at aubmindlab/araelectra-base-discriminator were not used when initializing ElectraForQuestionAnswering: ['discriminator_predictions.dense.weight', 'discriminator_predictions.dense.bias', 'discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense_prediction.bias']
- This IS expected if you are initializing ElectraForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForQuestionAnswering were not initialized from the model checkpoint at aubmindlab/araelectra-base-discriminator and are newly initialized: ['qa_outputs.weight'

134602752 42527232


ElectraForQuestionAnswering(
  (electra): ElectraModel(
    (embeddings): ElectraEmbeddings(
      (word_embeddings): Embedding(64000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): ElectraEncoder(
      (layer): ModuleList(
        (0): ElectraLayer(
          (attention): ElectraAttention(
            (self): ElectraSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): ElectraSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768

In [40]:
span_trained_model = span_train(QA_AraElectra,0, 4, span_optimizer,0.0, span_train_loader, val_loader, True, 'fourth')

Epoch 2:  50%|████▉     | 2616/5256 [24:29<23:41,  1.86it/s, loss=1.51]

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
{
  "exact": 31.754294638209267,
  "f1": 38.97888823055597,
  "total": 9605,
  "HasAns_exact": 58.001902949571836,
  "HasAns_f1": 71.20689276013132,
  "HasAns_total": 5255,
  "NoAns_exact": 0.04597701149425287,
  "NoAns_f1": 0.04597701149425287,
  "NoAns_total": 4350
}
ckp saved
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
{
  "exact": 32.181155648099946,
  "f1": 39.32540306155587,
  "total": 9

In [40]:
span_trained_model, span_optim , x, xx = load_ckp('Runs/AraElectraDecoupledAsquadv2/train/span/fourth/curr.pt', QA_AraElectra,span_optimizer)

In [41]:
max_metric = float(xx['HasAns_exact'])+1.5*float(xx['HasAns_f1'])
print(max_metric)

166.5424823233808


In [42]:
span_trained_model = span_train(span_trained_model,2, 4, span_optim,max_metric, span_train_loader, val_loader, True, 'fourth')

100%|██████████| 1201/1201 [05:50<00:00,  3.43it/s]


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
{
  "exact": 31.952108276939093,
  "f1": 39.32610496387188,
  "total": 9605,
  "HasAns_exact": 58.36346336822074,
  "HasAns_f1": 71.84152962473632,
  "HasAns_total": 5255,
  "NoAns_exact": 0.04597701149425287,
  "NoAns_f1": 0.04597701149425287,
  "NoAns_total": 4350
}
ckp saved
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
{
  "exact": 32.02498698594482,
  "f1": 39.26325553393223,
  "total": 960

## Evaluate Test Data

## Save Pretrained models

In [36]:
cls_model = ElectraForSequenceClassification.from_pretrained(model_name, num_labels = 2)
learning_rate = 3e-5
optimizer = torch.optim.AdamW(cls_model.parameters(), lr=learning_rate)
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
#criterion_span = nn.CrossEntropyLoss(reduction='none')
#cls_criterion = nn.CrossEntropyLoss()
cls_model.to(device)
cls_model , cls_optim, x, xx = load_ckp('Runs/AraElectraDecoupledAsquadv2/train/cls/third/best.pt', cls_model,optimizer)

Some weights of the model checkpoint at aubmindlab/araelectra-base-discriminator were not used when initializing ElectraForSequenceClassification: ['discriminator_predictions.dense.weight', 'discriminator_predictions.dense.bias', 'discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense_prediction.bias']
- This IS expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at aubmindlab/araelectra-base-discriminator and are newly initialized: 

In [39]:
cls_model.save_pretrained('AraElectra-ASQuADv2-CLS')
araelectra_tokenizer.save_pretrained('AraElectra-ASQuADv2-CLS')

('AraElectra-ASQuADv2-CLS/tokenizer_config.json',
 'AraElectra-ASQuADv2-CLS/special_tokens_map.json',
 'AraElectra-ASQuADv2-CLS/vocab.txt',
 'AraElectra-ASQuADv2-CLS/added_tokens.json')

In [43]:
qa_model = ElectraForQuestionAnswering.from_pretrained(model_name)
span_learning_rate = 3e-5
span_optimizer = torch.optim.AdamW(qa_model.parameters(), lr=span_learning_rate)
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
qa_model.to(device)
qa_model , qa_optim, x, xx = load_ckp('Runs/AraElectraDecoupledAsquadv2/train/span/fourth/best.pt', qa_model,span_optimizer)

Some weights of the model checkpoint at aubmindlab/araelectra-base-discriminator were not used when initializing ElectraForQuestionAnswering: ['discriminator_predictions.dense.weight', 'discriminator_predictions.dense.bias', 'discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense_prediction.bias']
- This IS expected if you are initializing ElectraForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForQuestionAnswering were not initialized from the model checkpoint at aubmindlab/araelectra-base-discriminator and are newly initialized: ['qa_outputs.weight'

In [44]:
qa_model.save_pretrained('AraElectra-ASQuADv2-QA')
araelectra_tokenizer.save_pretrained('AraElectra-ASQuADv2-QA')

('AraElectra-ASQuADv2-QA/tokenizer_config.json',
 'AraElectra-ASQuADv2-QA/special_tokens_map.json',
 'AraElectra-ASQuADv2-QA/vocab.txt',
 'AraElectra-ASQuADv2-QA/added_tokens.json')

## Evaluate on Test Data

In [33]:
qa_model = ElectraForQuestionAnswering.from_pretrained('AraElectra-ASQuADv2-QA')
cls_model = ElectraForSequenceClassification.from_pretrained('AraElectra-ASQuADv2-CLS',num_labels=2)
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
qa_model.to(device)
cls_model.to(device)

ElectraForSequenceClassification(
  (electra): ElectraModel(
    (embeddings): ElectraEmbeddings(
      (word_embeddings): Embedding(64000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): ElectraEncoder(
      (layer): ModuleList(
        (0): ElectraLayer(
          (attention): ElectraAttention(
            (self): ElectraSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): ElectraSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm

In [34]:
cls_res_dict = cls_eval(cls_model, test_loader,'Runs/AraElectraDecoupledAsquadv2/train/cls/third/test-eval',None)

In [35]:
cls_res_dict

{'acc': 84.53930020332336, 'train_loss': None}

In [37]:
cls_preds_dict = dict()
with open('Runs/AraElectraDecoupledAsquadv2/train/cls/third/test-eval/preds.txt','r') as f:
    for line in f:
        line = line.strip()
        elements = line.split(',')
        cls_preds_dict[elements[0]] = float(elements[1])
print(len(cls_preds_dict))

9606


In [43]:
imd_preds, script_preds = get_raw_preds(test_loader, qa_model,test_ids_to_idx,test_offset,asquad_test_contexts, 30, 10)

100%|██████████| 1201/1201 [05:47<00:00,  3.45it/s]


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Traceback (most recent call last):
  File "evaluatev2.py", line 295, in <module>
    main()
  File "evaluatev2.py", line 240, in main
    with open(OPTS.na_prob_file) as f:
FileNotFoundError: [Errno 2] No such file or directory: 'Runs/AraElectraDecoupledAsquadv2/train/span/second/preds/na_probs.json'


FileNotFoundError: [Errno 2] No such file or directory: 'Runs/AraElectraDecoupledAsquadv2/train/span/fourth/test-eval/res.csv'

In [48]:
qa_result_dict = get_preds2(script_preds, cls_preds_dict,'Data/asquadv2-test.json','Runs/AraElectraDecoupledAsquadv2/train/span/fourth/test-eval' )

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
{
  "exact": 65.11555277951281,
  "f1": 71.49042547237256,
  "total": 9606,
  "HasAns_exact": 56.14535768645358,
  "HasAns_f1": 67.79623803036668,
  "HasAns_total": 5256,
  "NoAns_exact": 75.95402298850574,
  "NoAns_f1": 75.95402298850574,
  "NoAns_total": 4350,
  "best_exact": 67.7597334998959,
  "best_exact_thresh": 0.12791500985622406,
  "best_f1": 73.3514620564771,
  "best_f1_thresh": 0.20527765154838562
}


In [47]:
def get_preds2(total_preds, no_probs_preds,data_path, log_path):
    preds_path = os.path.join(log_path, 'preds')
    if not os.path.exists(preds_path):
        os.mkdir(preds_path)
    no_probs_path = os.path.join(preds_path, 'na_probs.json')
    text_preds_path = os.path.join(preds_path, 'preds.json')
    jsonString = json.dumps(total_preds)
    jsonFile = open(text_preds_path, "w")
    jsonFile.write(jsonString)
    jsonFile.close()
    if no_probs_preds is not None:
        jsonString = json.dumps(no_probs_preds)
        jsonFile = open(no_probs_path, "w")
        jsonFile.write(jsonString)
        jsonFile.close()
        #!python evaluatev2.py data_path text_preds_path electra --na-prob-file no_probs_path --na-prob-thresh 0.4 --out-file log_path
        #os.system(f"python evaluatev2.py {data_path} {text_preds_path} electra --na-prob-file {no_probs_path} --na-prob-thresh 0.5 --out-file {log_path}")
        !/anaconda/envs/azureml_py38/bin/python3 evaluatev2.py Data/asquadv2-test.json Runs/AraElectraDecoupledAsquadv2/train/span/fourth/test-eval/preds/preds.json electra --na-prob-file Runs/AraElectraDecoupledAsquadv2/train/span/fourth/test-eval/preds/na_probs.json  --na-prob-thresh 0.5  
    return 1